In [44]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [45]:
# Copy the parent path of the folder that contains this notebook using the file navigation on the left:
folderPath = '/content/drive/My Drive/CIS 581 - Computer Vision and Computational Photography/Test'

In [46]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from scipy import signal
from PIL import Image
import argparse
import math


# Tests and Visualization

In [47]:
'''
    Used to check if the Image and Edge shapes are correct before proceeding with display
  '''
def Test_script(I, E):
    test_pass = True

    # E should be 2D matrix
    if E.ndim != 2:
      print('ERROR: Incorrect Edge map dimension! \n')
      print(E.ndim)
      test_pass = False
    # end if

    # E should have same size with original image
    nr_I, nc_I = I.shape[0], I.shape[1]
    nr_E, nc_E = E.shape[0], E.shape[1]

    if nr_I != nr_E or nc_I != nc_E:
      print('ERROR: Edge map size has changed during operations! \n')
      test_pass = False
    # end if

    # E should be a binary matrix so that element should be either 1 or 0
    numEle = E.size
    numOnes, numZeros = E[E == 1].size, E[E == 0].size

    if numEle != (numOnes + numZeros):
      print('ERROR: Edge map is not binary one! \n')
      test_pass = False
    # end if

    if test_pass:
      print('Shape Test Passed! \n')
    else:
      print('Shape Test Failed! \n')

    return test_pass

In [48]:
'''
  Derivatives visualzation function
'''
def visDerivatives(I_gray, Mag, Magx, Magy):
    fig, (Ax0, Ax1, Ax2, Ax3) = plt.subplots(1, 4, figsize = (20, 8))

    Ax0.imshow(Mag, cmap='gray', interpolation='nearest')
    Ax0.axis('off')
    Ax0.set_title('Gradient Magnitude')

    Ax1.imshow(Magx, cmap='gray', interpolation='nearest')
    Ax1.axis('off')
    Ax1.set_title('Gradient Magnitude (x axis)')
    
    Ax2.imshow(Magy, cmap='gray', interpolation='nearest')
    Ax2.axis('off')
    Ax2.set_title('Gradient Magnitude (y axis)')

    # plot gradient orientation
    Mag_vec = Mag.transpose().reshape(1, Mag.shape[0] * Mag.shape[1]) 
    hist, bin_edge = np.histogram(Mag_vec.transpose(), 100)

    ind_array = np.array(np.where( (np.cumsum(hist).astype(float) / hist.sum()) < 0.95))
    thr = bin_edge[ind_array[0, -1]]

    ind_remove = np.where(np.abs(Mag) < thr)
    Magx[ind_remove] = 0
    Magy[ind_remove] = 0

    X, Y = np.meshgrid(np.arange(0, Mag.shape[1], 1), np.arange(0, Mag.shape[0], 1))
    Ori = np.arctan2(Magy, Magx)
    ori = Ax3.imshow(Ori, cmap='hsv')
    Ax3.axis('off')
    Ax3.set_title('Gradient Orientation')
    fig.colorbar(ori, ax=Ax3, )
    


'''
  Edge detection result visualization function
'''
def visCannyEdge(Im_raw, M, E):
    # plot image
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize = (12, 12))

    # plot original image
    ax0.imshow(Im_raw)
    ax0.axis("off")
    ax0.set_title('Raw image')

    # plot edge detection result
    ax1.imshow(M, cmap='gray', interpolation='nearest')
    ax1.axis("off")
    ax1.set_title('Non-Max Suppression Result')

    # plot original image
    ax2.imshow(E, cmap='gray', interpolation='nearest')
    ax2.axis("off") 
    ax2.set_title('Canny Edge Detection')

# Functions

In [49]:
''' Take in the Magnitude, Non-maximum suppression and orientation matrixes, combine using high and low thresholds to link edges and return Canny image matrix
        Compute gradient information of the input grayscale image
        - Input M: H x W matrix as boolean non-maximum suppression matrix
        - Input Mag: H x W matrix represents the magnitude of derivatives
        - Input Ori: H x W matrix represents the orientation of derivatives
        - Input low: low threshold for edge hysteresis
        - Input high: high threshold for edge hysteresis
        - Output EdgeMap: H x W matrix represents the edges detected through Canny Edge Detection once edge-hysteresis is complete
  '''
def edgeLink(M, Mag, Ori, low, high):
    # double threshold, focus on these in uncertain points
    # suppress the magnitude where edgemap is false(false means suppressed in NMS step)
    Mag_suppressed = np.where(M, Mag, 0)
    Uncertain = np.logical_and(low < Mag_suppressed, Mag_suppressed < high)
    # initial EdgeMap with high points, later add linked uncertain points to it
    EdgeMap = Mag_suppressed >= high
    # compute the edge orientation(perpendicular to the orientation direction)
    EdgeOri = Ori + np.pi / 2
    # EdgeOri = np.where(EdgeOri < 0, EdgeOri + np.pi, EdgeOri) # add pi?

    # find neighbor coordinations in the edge direction
    nr, nc = Uncertain.shape
    x, y = np.meshgrid(np.arange(nc), np.arange(nr))
    Neighbor1_x = np.clip(x + np.cos(EdgeOri), 0, nc - 1)
    Neighbor1_y = np.clip(y + np.sin(EdgeOri), 0, nr - 1)

    Neighbor2_x = np.clip(x - np.cos(EdgeOri), 0, nc - 1)
    Neighbor2_y = np.clip(y - np.sin(EdgeOri), 0, nr - 1)
    done = False

    # loop until uncertain points don't change anymore
    while not done:
        # interpolate the magnitude of neighbor
        Neighbor1 = interp2(Mag_suppressed, Neighbor1_x, Neighbor1_y)
        Neighbor2 = interp2(Mag_suppressed, Neighbor2_x, Neighbor2_y)
        
        # if its neighbor is above high threshold, then it has a strong point neighbor
        # copy the magnitude and remove it from uncertain points
        nearStrongPoints = np.logical_or(Neighbor1 >= high,
                                         Neighbor2 >= high)
        toUpdate = np.logical_and(Uncertain, nearStrongPoints)
        # update the magnitude of uncertain-near-strong points
        Mag_suppressed = np.where(toUpdate, np.maximum(Neighbor1, Neighbor2), Mag_suppressed)
        Uncertain[toUpdate] = False
        EdgeMap[toUpdate] = True
        if not np.any(toUpdate):
            done = True
            
    return EdgeMap

In [50]:
''' Interpolate a matrix between two others
'''
def interp2(v, xq, yq):
	if len(xq.shape) == 2 or len(yq.shape) == 2:
		dim_input = 2
		q_h = xq.shape[0]
		q_w = xq.shape[1]
		xq = xq.flatten()
		yq = yq.flatten()

	h = v.shape[0]
	w = v.shape[1]
	if xq.shape != yq.shape:
		raise 'query coordinates Xq Yq should have same shape'

	x_floor = np.floor(xq).astype(np.int32)
	y_floor = np.floor(yq).astype(np.int32)
	x_ceil = np.ceil(xq).astype(np.int32)
	y_ceil = np.ceil(yq).astype(np.int32)

	x_floor[x_floor < 0] = 0
	y_floor[y_floor < 0] = 0
	x_ceil[x_ceil < 0] = 0
	y_ceil[y_ceil < 0] = 0

	x_floor[x_floor >= w-1] = w-1
	y_floor[y_floor >= h-1] = h-1
	x_ceil[x_ceil >= w-1] = w-1
	y_ceil[y_ceil >= h-1] = h-1

	v1 = v[y_floor, x_floor]
	v2 = v[y_floor, x_ceil]
	v3 = v[y_ceil, x_floor]
	v4 = v[y_ceil, x_ceil]

	lh = yq - y_floor
	lw = xq - x_floor
	hh = 1 - lh
	hw = 1 - lw

	w1 = hh * hw
	w2 = hh * lw
	w3 = lh * hw
	w4 = lh * lw

	interp_val = v1 * w1 + w2 * v2 + w3 * v3 + w4 * v4

	if dim_input == 2:
		return interp_val.reshape(q_h, q_w)
	return interp_val

In [51]:
'''
  Convert RGB image to gray one manually
  - Input I_rgb: 3-dimensional rgb image
  - Output I_gray: 2-dimensional grayscale image
'''
def rgb2gray(I_rgb):
    r, g, b = I_rgb[:, :, 0], I_rgb[:, :, 1], I_rgb[:, :, 2]
    I_gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return I_gray

In [52]:
def findDerivatives(I_gray):
    '''
        Compute gradient information of the input grayscale image
        - Input I_gray: H x W matrix as image
        - Output Mag: H x W matrix represents the magnitude of derivatives
        - Output Magx: H x W matrix represents the magnitude of derivatives along x-axis
        - Output Magy: H x W matrix represents the magnitude of derivatives along y-axis
        - Output Ori: H x W matrix represents the orientation of derivatives
    '''
    dx = np.matrix('1 0 -1; 2 0 -2; 1 0 -1') #Gradient along x-axis
    dy = np.matrix('1 2 1; 0 0 0; -1 -2 -1') #Gradient along y-axis
    G = np.matrix('2 4 5 4 2; 4 9 12 9 4; 5 12 15 12 5; 4 9 12 9 4; 2 4 5 4 2') #Gaussian matrix
    G = G / 159 
    Gx = signal.convolve2d(G, dx, mode = 'same', fillvalue = 0) #X-derivative of Gaussian Matrix
    Gy = signal.convolve2d(G, dy, mode = 'same', fillvalue = 0) #Y-derivative of Gaussian Matrix
    Magx = signal.convolve2d(I_gray, Gx, mode = 'same', fillvalue = 0) #Magnitude of pixel change in x-direction
    Magy = signal.convolve2d(I_gray, Gy, mode = 'same', fillvalue = 0) #Magnitude of pixel change in Y-direction
    Mag = np.sqrt(np.square(Magx) + np.square(Magy)) #Magnitude matrix using SRSS 9Square-Root Sum of Squares)
    Ori = np.arctan2(Magy,Magx) #Gradient orientation Matrix
    return Mag, Magx, Magy, Ori

In [53]:
# Test cell
I = plt.imread(folderPath + '/Images/I1.jpg')
Mag, Magx, Magy, Ori = findDerivatives(I)
assert np.allclose(Mag, np.load(folderPath + '/Code/Mag.npy'))
assert np.allclose(Magx, np.load(folderPath + '/Code/Magx.npy'))
assert np.allclose(Magy, np.load(folderPath + '/Code/Magy.npy'))
assert np.allclose(Ori, np.load(folderPath + '/Code/Ori.npy'))

In [54]:
def nonMaxSup(Mag, Ori):
    '''
        Find local maximum edge pixel using NMS along the line of the gradient
        - Input Mag: H x W matrix represents the magnitude of derivatives
        - Input Ori: H x W matrix represents the orientation of derivatives
        - Output M: H x W binary matrix represents the edge map after non-maximum suppression
    '''
    nr, nc = Mag.shape
    x, y = np.meshgrid(np.arange(nc), np.arange(nr))
    # getting neighbor in the oritention direction
    neighbor1_x = np.clip(x + np.cos(Ori), 0, nc - 1)
    neighbor1_y = np.clip(y + np.sin(Ori), 0, nr - 1)
    # using interpolation to get neighbor
    neighbor1 = interp2(Mag, neighbor1_x, neighbor1_y)
    # getting neighbor in the opposite of the oritention direction
    # TODO: get neighbor 2
    neighbor2_x = np.clip(x - np.cos(Ori), 0, nc - 1) #clip in the opposite direction of neighbor one
    neighbor2_y = np.clip(y - np.sin(Ori), 0, nr - 1)
    # using interpolation to get neighbor
    neighbor2 = interp2(Mag, neighbor2_x, neighbor2_y) #interpolate between values to get neighbor
    # TODO: perform NMS
    M = np.zeros((nr, nc), dtype = bool) #initialize nonMaxSuppression matrix to all false
    for y in range(0, nr):
      for x in range(0, nc):
        if Mag[y,x] >= neighbor1[y,x] and Mag[y,x] >= neighbor2[y,x]: #if the value of the magnitude is larger than both its neighbors then set the value to true
          M[y,x] = True
    return M


In [55]:
# Test cell
Mag = np.array([[15, 8, 7],
                [3, 10, 9],
                [6, 4, 20],
               ])
Ori = np.array([[0, np.pi/2, 3*np.pi/2],
                [np.pi, 5*np.pi/4, 0],
                [np.pi/4, 3*np.pi/4, np.pi/4],
                ])
M = nonMaxSup(Mag, Ori)
res = np.array([[ True, False, False],
       [False, False, False],
       [ True, False,  True]])
assert M.dtype == bool
assert M.shape == Mag.shape
assert np.allclose(M, res)

In [56]:
def ThresholdGenerator(Mag, HighThresholdRatio = 0.15, LowThresholdRatio = 0.4):
  ''' Determine high and low thresholds using Magnitude matrix high/low threshold ratios
  '''
  highThreshold = np.max(Mag) * HighThresholdRatio 
  lowThreshold = highThreshold * LowThresholdRatio
  return lowThreshold, highThreshold

In [57]:
''' Perform Canny Edge Detection of Input image
        - Input I: H x W matrix as image
        - Output E: Edge image linked using non-maximum suppression and edge hysteresis
  '''
def cannyEdge(I):
    # convert RGB image to gray color space
    im_gray = rgb2gray(I)
    # return magnitude and orientation matrices
    Mag, Magx, Magy, Ori = findDerivatives(im_gray)
    #perform non-maximum suppression
    M = nonMaxSup(Mag, Ori)
    # determine high and low thresholds
    low, high = ThresholdGenerator(Mag)
    # link edges together
    E = edgeLink(M, Mag, Ori, low, high)

    # only when test passed that can show all results
    if Test_script(im_gray, E):
        # visualization results
        visDerivatives(im_gray, Mag, Magx, Magy)
        visCannyEdge(im_gray, M, E)

        plt.show()

    return E

Tune the threshold for each images 

In [58]:
 # list all image names
 os.listdir(folderPath + '/Images')

['OldMan.jpg',
 'Race.jpg',
 'Chapel.jpg',
 'Bridge2.jpg',
 'I1.jpg',
 'Zebra.jpg',
 'Bird3.jpg',
 'Building1.jpg',
 'Buildings.jpg',
 'Plane.jpg',
 'Chess.jpg',
 'StarFish.jpg',
 'Panda.JPG',
 'Boat.jpg',
 'Bird2.jpg',
 'BurjAlArab.jpg',
 'Rocks.jpg',
 'Swan.jpg',
 'Cars.jpg',
 'ForestRoad.jpg',
 'Bird1.jpg',
 'WallChina.jpg',
 'Kremlin.jpg',
 'Island1.jpg',
 'Pattern1.jpg',
 'Lion_HalfBlurred.jpg',
 'Painting.jpg',
 'BlueBird.jpg',
 'RockSea.jpg',
 'Flower.jpg',
 'Butterfly.jpg',
 'Bridge.jpg',
 'BurjKhalifa.jpg',
 'GeniewAladdin.jpg']

# Fill in all tuned threshold to generate edge detection results


In [59]:
# keep results for all images
image_folder = folderPath + "/Images"
save_folder = folderPath + "/Results"
# generate results one by one
for filename in os.listdir(image_folder):
    # read in image 
    im_path = os.path.join(image_folder, filename)
    I = np.array(Image.open(im_path).convert('RGB'))

    #perform canny edge detection
    E = cannyEdge(I)

    #save image in results folder
    pil_image = Image.fromarray(E.astype(np.uint8) * 255).convert('L')

    pil_image.save(os.path.join(save_folder, "{}_Result.png".format(filename.split(".")[0])))

Output hidden; open in https://colab.research.google.com to view.